# Aprendizado de Máquina Automatizado (AutoML)

Aprendizado de Máquina Automatizado (AutoML) refere-se ao processo de automatização do fluxo completo de aplicação de aprendizado de máquina a problemas do mundo real. Isso inclui tarefas como pré-processamento de dados, engenharia de características, seleção de modelos, ajuste de hiperparâmetros e implantação de modelos, tudo isso sem necessitar de uma intervenção manual extensa.

Existem frameworks e bibliotecas que facilitam a comparação e avaliação de diferentes algoritmos de aprendizado de máquina. Eles oferecem ferramentas para automatizar o processo de treino, teste, validação e seleção do melhor modelo. 

## Benefícios do AutoML

### Economia de Tempo

O desenvolvimento manual de modelos exige tempo e esforço significativos em tarefas como pré-processamento de dados, engenharia de características e ajuste de hiperparâmetros. O AutoML simplifica essas tarefas, permitindo que os usuários se concentrem em resolver problemas de maior nível e inovar.

Estudos mostram que o AutoML pode reduzir o tempo gasto no desenvolvimento de modelos em até 90%, possibilitando uma iteração e experimentação mais rápidas.

### Acessibilidade

As plataformas de AutoML abstraem as complexidades do aprendizado de máquina, tornando-o acessível a usuários sem expertise especializada em ciência de dados ou programação.

O AutoML capacita especialistas de domínio, analistas de negócios e pesquisadores a aproveitarem o poder do aprendizado de máquina sem a necessidade de conhecimento técnico extenso.

### Escalabilidade

Os frameworks de AutoML são projetados para lidar com grandes volumes de dados e ambientes de computação distribuída, permitindo uma escalabilidade perfeita para acomodar volumes crescentes de dados e demandas computacionais.

O AutoML da Google, por exemplo, pode escalar para milhares de nós e processar petabytes de dados, tornando-o adequado para aplicações em nível empresarial.

## Limitações do AutoML

### Falta de Transparência

O AutoML automatiza muitos aspectos do processo de desenvolvimento de modelos, o que pode obscurecer o funcionamento interno dos modelos resultantes. Essa falta de transparência pode tornar desafiador entender e interpretar as decisões dos modelos.

Algumas plataformas de AutoML geram modelos complexos que são difíceis de interpretar, levantando preocupações sobre responsabilidade e confiabilidade, especialmente em indústrias regulamentadas como saúde e finanças.

### Personalização Limitada

Embora o AutoML ofereça conveniência e eficiência, ele pode não fornecer o mesmo nível de personalização e ajuste fino que o desenvolvimento manual de modelos.

Deve-se ponderar os benefícios da automação contra a necessidade de controle detalhado sobre a arquitetura do modelo e os hiperparâmetros.


<div style="border: 2px solid #4CAF50; padding: 15px; border-radius: 10px; background-color: #DFF2BF; color: #4F8A10; font-family: Arial;">
AutoML e ML Manual têm suas próprias vantagens e desvantagens, e a melhor abordagem depende do contexto específico e dos requisitos do problema.

AutoML é mais rápido e exige menos conhecimento especializado, mas pode produzir modelos menos precisos ou menos interpretáveis. É uma boa opção para problemas simples ou quando se tem tempo ou recursos limitados.

ML manual requer mais expertise e tempo, mas permite um maior controle sobre o modelo e frequentemente resulta em soluções mais precisas e interpretáveis. É uma boa opção para problemas mais complexos ou quando há mais recursos e tempo disponíveis.

</div>

.

In [ ]:
!pip install pycaret

In [ ]:
!pip install tpot

In [ ]:
!pip install h2o

In [ ]:
!pip install seaborn

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import r2_score

import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import joblib
from joblib import dump


In [ ]:
models_result = []

In [ ]:
df_normalizado = pd.read_csv("../data/processed/predicao_cargas_trabalho_normalizado.csv")

X = df_normalizado.loc[:, df_normalizado.columns != 'casos_pendentes']
y = df_normalizado["casos_pendentes"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>PyCaret</h3>
</div>

PyCaret é uma biblioteca de AutoML que fornece uma interface simplificada para comparação de modelos, tuning de hiperparâmetros e seleção de características. 

In [ ]:
from pycaret.regression import setup, compare_models, create_model, get_metrics, predict_model, evaluate_model, finalize_model, save_model, load_model

data_train = pd.concat([X_train, y_train], axis=1)
data_test = pd.concat([X_test, y_test], axis=1)

# Carregando o dataset de treino no PyCaret
reg = setup(data=data_train, target='casos_pendentes', session_id=123,  verbose=False)

# Comparando todos os modelos para escolher o melhor
best_model = compare_models()


In [ ]:
best_model

In [ ]:
evaluate_model(best_model)

In [ ]:
predictions = predict_model(best_model, data=data_test)

In [ ]:
predictions

In [ ]:
mae_func = get_metrics().loc['mae', 'Score Function']
mse_func = get_metrics().loc['mse', 'Score Function']
rmse_func = get_metrics().loc['rmse', 'Score Function']
r2_func = get_metrics().loc['r2', 'Score Function']

mae = mae_func(predictions['prediction_label'], predictions['casos_pendentes'])
mse = mse_func(predictions['prediction_label'], predictions['casos_pendentes'])
r2 =  r2_func(predictions['prediction_label'], predictions['casos_pendentes'])
rmse = np.sqrt(mse)

metric_model = {
    'MODEL': str(best_model)+'+PyCaret ',
    'MAE': mae,
    'MSE': mse,
    'RMSE': rmse,
    'R2': r2
}

models_result.append(metric_model)

In [ ]:
sns.histplot((predictions['casos_pendentes']-predictions['prediction_label']),bins=50);

In [ ]:
final_model = finalize_model(best_model)
save_model(final_model, '../models/automl/modelPyCaret')

# Carregando o modelo salvo
#loaded_model = load_model('./modelo/automl/modelPyCaret')

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>TPOT (Tree-based Pipeline Optimization Tool)</h3>
</div>

Usa algoritmos genéticos para automatizar o processo de construção de pipelines de aprendizado de máquina. Ele explora várias combinações de pré-processamento, seleção de características e algoritmos de aprendizado.

In [ ]:
from tpot import TPOTRegressor

modelTPOT = TPOTRegressor(generations=5, population_size=20, verbosity=2)
modelTPOT.fit(X_train, y_train)

predictions = modelTPOT.predict(X_test)

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))
print('R2:', r2_score(y_test, predictions))

In [ ]:
# Exportar o pipeline
#modelTPOT.export('../models/automl/modelTPOT.py')

best_model = modelTPOT.fitted_pipeline_
dump(best_model, '../models/automl/modelTPOT.joblib')

In [ ]:
sns.histplot((y_test-predictions),bins=50);

In [ ]:
pipeline = modelTPOT.fitted_pipeline_
last_step_name, last_step = pipeline.steps[-1]
mode_name = last_step.__class__.__name__

In [ ]:
metric_model = {
    'MODEL': mode_name+'+TPOTRegressor',
    'MAE': metrics.mean_absolute_error(y_test, predictions),
    'MSE': metrics.mean_squared_error(y_test, predictions),
    'RMSE': np.sqrt(metrics.mean_squared_error(y_test, predictions)),
    'R2': r2_score(y_test, predictions)
}

models_result.append(metric_model)

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>H2OAutoML</h3>
</div>

H2O.ai é uma plataforma de AutoML que oferece uma ampla gama de modelos de aprendizado de máquina, incluindo regressão, classificação, e métodos de ensemble. Ele permite comparar automaticamente vários modelos e otimizar seus hiperparâmetros. Tem ssuporte a modelos distribuídos, de grande escala e integração com Spark e outras plataformas.

In [ ]:
import h2o
from h2o.automl import H2OAutoML

h2o.init()

train = h2o.H2OFrame(pd.concat([X_train, y_train], axis=1))
test = h2o.H2OFrame(pd.concat([X_test, y_test], axis=1))

In [ ]:
X_train_ = X_train.columns.tolist() 
y_train_ = y_train.name           

aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=X_train_, y=y_train_, training_frame=train)

# Obtém o melhor modelo
modelH2O = aml.leader

# Exibe a tabela de desempenho dos modelos
leaderboard = aml.leaderboard
print(leaderboard)

In [ ]:
predictions = modelH2O.predict(test)

# Converte as previsões para um DataFrame pandas
predictions_df = h2o.as_list(predictions, use_pandas=True)
y_pred = predictions_df['predict']

In [ ]:
# Converte o conjunto de teste para um DataFrame pandas para comparação
test_df = h2o.as_list(test, use_pandas=True)
y_test_ = test_df["casos_pendentes"] 

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test_, y_pred))
print('MSE:', metrics.mean_squared_error(y_test_, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test_, y_pred)))
print('R2:', r2_score(y_test_, y_pred))

In [ ]:
model_path = h2o.save_model(model=modelH2O, path="../models/automl", force=True)

In [ ]:
sns.histplot((y_test_-y_pred),bins=50);

In [ ]:
metric_model = {
    'MODEL': modelH2O.model_id+'+h2o',
    'MAE': metrics.mean_absolute_error(y_test_, y_pred),
    'MSE': metrics.mean_squared_error(y_test_, y_pred),
    'RMSE': np.sqrt(metrics.mean_squared_error(y_test_, y_pred)),
    'R2': r2_score(y_test_, y_pred)
}

models_result.append(metric_model)

In [ ]:
# Encerra a sessão H2O
h2o.shutdown(prompt=False)